# NMR vs EPR vs muSR

Visualizing differences in ZFS signatures for different techniques (NMR, EPR, muSR) in transverse field (TF) experiments using S=1/2, I=1/2 system

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly.colors as pc
import xarray as xr
import netCDF4 as nc
from utils import stick_spectrum
from scipy.io import loadmat, savemat

import Python.plot_settings
pio.templates.default = "DemonLab"


In [ ]:
import plotly.graph_objects as go

def add_doubleheaded_arrow(
    fig,
    xrange,
    yrange,
    x1,
    x2=None,
    y=None,
    label="",
    y2=None,
    orientation="h",
    label_offset=-2,
    font_size=20,
    line_width=2,
    head_size=14,
    arrow_indent=0.05,
):
    """
    Adds a horizontal or vertical double-headed arrow with a label to a Plotly figure,
    using triangle markers for arrowheads.

    Parameters
    ----------
    fig : plotly.graph_objects.Figure
        Target figure.
    range : tuple
    Range of arrow points.
    x1, x2 : float
        For horizontal arrows: start and end x positions.
        For vertical arrows: x1 is the x position (x2 ignored).
    y, y2 : float
        For vertical arrows: start and end y positions.
        For horizontal arrows: y is given in paper coordinates (0–1).
    orientation : {"h", "v"}
        Direction of the arrow.
    label : str
        Text label to display at the midpoint.
    label_offset : float
        Offset of the label in pixels (vertical for horizontal arrows, horizontal for vertical).
    font_size : int
        Label font size.
    line_width : float
        Width of the arrow shaft.
    head_size : float
        Size of the triangle arrowheads.
    """

    if orientation == "h":
        if x2 is None or y is None:
            raise ValueError("x2 and y must be provided for horizontal arrows.")

        y_shift = label_offset * abs(yrange[1] - yrange[0])
        x_shift = arrow_indent * abs(xrange[1] - xrange[0])

        # Shaft
        fig.add_shape(
            type="line",
            x0=x1+x_shift, y0=y,
            x1=x2-x_shift, y1=y,
            line=dict(color="black", width=line_width),
            xref="x", yref='y',
            showlegend=False,
        )

        # Arrowheads
        fig.add_trace(go.Scatter(
            x=[x1+x_shift, x2-x_shift],
            y=[y, y],
            mode="markers",
            marker_symbol=["triangle-left", "triangle-right"],
            marker_size=head_size,
            marker_color="black",
            marker_line_color="black",
            marker_line_width=1,
            hoverinfo="skip",
            xaxis="x",
            yaxis="y",
            showlegend=False,
        ))

        # Label
        fig.add_annotation(
            x=(x1 + x2) / 2,
            y=y+y_shift,
            text=label,
            showarrow=False,
            font=dict(size=font_size),
            xref="x",
            yref="y",
        )

    elif orientation == "v":
        if y2 is None or x1 is None:
            raise ValueError("x1 and y2 must be provided for vertical arrows.")

        # Shaft
        fig.add_shape(
            type="line",
            x0=x1, y0=y,
            x1=x1, y1=y2,
            line=dict(color="black", width=line_width),
            xref="x", yref="y",
            showlegend=False,
        )

        # Arrowheads
        fig.add_trace(go.Scatter(
            x=[x1, x1],
            y=[y, y2],
            mode="markers",
            marker_symbol=["triangle-up", "triangle-down"],
            marker_size=head_size,
            marker_color="black",
            marker_line_color="black",
            marker_line_width=1,
            hoverinfo="skip",
            showlegend=False,
        ))

        # Label
        fig.add_annotation(
            x=x1,
            y=(y + y2) / 2,
            xshift=label_offset,
            text=label,
            showarrow=False,
            font=dict(size=font_size),
            xref="x",
            yref="y",
            showlegend=False,
        )

    else:
        raise ValueError("orientation must be either 'h' or 'v'.")


def get_concoluted_spectrum(freq, amps, freq_axis, gamma):
    """
    freq: array of peak frequencies
    amps: array of peak amplitudes
    freq_axis: frequency axis for plotting
    gamma: FWHM of each Lorentzian peak
    """
    def lorentzian(x, x0, amp, gamma):
        return amp * (0.5*gamma)**2 / ((x - x0)**2 + (0.5*gamma)**2)

    spectrum = np.zeros_like(freq_axis)
    for f, a in zip(freq, amps):
        if np.isfinite(f) and np.isfinite(a):
            spectrum += lorentzian(freq_axis, f, a, gamma)
    return spectrum

# Problem of MathJax rendering, it can't go beyond font size 20
# therefore use r'$\Large ...$' for larger symbols
def create_ZF_base_figure():
    fig = go.Figure()
    fig.update_layout(
        xaxis_title='𝜈',    # or use r'$\Large \nu$'
        yaxis_title='Amplitude',
        xaxis=dict(
            showticklabels=False,   # remove x ticks
            showgrid=False,
            ticks='',               # remove tick marks
            title_standoff=8,        # decrease distance between x label and axis
            minor=dict(ticks='', showgrid=False)  # remove minor ticks and minor grid
        ),
        margin=dict(l=90, r=20, t=20, b=40),
        height=400, width=600,
    )
    return fig


## Zero Field Splitting


### NMR

#### Only dipolar coupling of D_parallel = 2 MHz
expect 2 peaks with a splitting of 3D

In [ ]:
xarr = xr.load_dataset('Data/TF_NMR_dipolar_cc.nc')
theta = 0
B = 0

freq = xarr['frequencies'].sel(B=B, theta=theta).values * 1e3 # Convert to GHz
amps = xarr['amplitudes'].sel(B=B, theta=theta).values

# Define the frequency axis for plotting
# freq_min, freq_max = -2, 2
freq_min, freq_max = -4, 4
n_points = 8000
freq_axis = np.linspace(freq_min, freq_max, n_points)
gamma = 0.01  # MHz

spectrum = get_concoluted_spectrum(freq, amps, freq_axis, gamma)

xrange = [freq_min, freq_max]
yrange = [-0.02, 0.53]

# Plot
fig = create_ZF_base_figure()
fig.add_trace(go.Scatter(x=freq_axis, y=spectrum, mode='lines', line=dict(color='black', width=2), showlegend=False))
add_doubleheaded_arrow(fig, xrange, yrange, freq[0], freq[1], 0.45, r'$\Large \frac{3}{2}D_\parallel$', label_offset=-0.06, font_size=20, arrow_indent=0.02)
fig.update_layout(yaxis_range=yrange)

fig.show(renderer='browser')
# fig.write_image('../../Figures/Manuscript/TF_simulations/NMR_dipolar_ZFS.pdf')


#### Only anisotropic hyperfine coupling of A_iso = 2 MHz

In [ ]:
xarr = xr.load_dataset('Data/TF_NMR_isotropic_cc.nc')
# print(ds)
theta = 0
B = 0

freq = xarr['frequencies'].sel(B=B, theta=theta).values * 1e3 # Convert to GHz
amps = xarr['amplitudes'].sel(B=B, theta=theta).values

# Define the frequency axis for plotting
freq_min, freq_max = -3, 3
n_points = 8000
freq_axis = np.linspace(freq_min, freq_max, n_points)
gamma = 0.01  # MHz

spectrum = get_concoluted_spectrum(freq, amps, freq_axis, gamma)

# Plot
fig = create_ZF_base_figure()
fig.add_trace(go.Scatter(x=freq_axis, y=spectrum, mode='lines', line=dict(color='black', width=2)))

fig.show()
# fig.write_image('../../Figures/Manuscript/TF_simulations/NMR_iso_ZFS.pdf')


### muSR

#### Only dipolar coupling of D_parallel = 2 MHz
expect 4 peaks with 1/2 D splitting, 2 peaks in middle are SQE transitions which are invisible in NMR

In [ ]:
xarr = xr.load_dataset('Data/TF_muSR_dipolar_cc.nc')
# print(ds)
theta = 0
B = 0

freq = xarr['frequencies'].sel(B=B, theta=theta).values * 1e3 # Convert to GHz
amps = xarr['amplitudes'].sel(B=B, theta=theta).values

# Define the frequency axis for plotting
freq_min, freq_max = -2, 2
n_points = 8000
freq_axis = np.linspace(freq_min, freq_max, n_points)
gamma = 0.01  # MHz

spectrum = get_concoluted_spectrum(freq, amps, freq_axis, gamma)

# Plot
fig = create_ZF_base_figure()
fig.add_trace(go.Scatter(x=freq_axis, y=spectrum, mode='lines', line=dict(color='black', width=2)))
add_doubleheaded_arrow(fig, freq[0], freq[2], 0.8, r'$\Large \frac{1}{2}D_\parallel$', label_offset=-2, font_size=20)

fig.show(renderer='browser')
# fig.write_image('../../Figures/Manuscript/TF_simulations/muSR_dipolar_ZFS.pdf')


#### Only isotropic hyperfine coupling of A_iso = 2 MHz

3 peaks with A splitting

In [ ]:
xarr = xr.load_dataset('Data/TF_muSR_isotropic_cc.nc')
# print(ds)
theta = 0
B = 0

freq = xarr['frequencies'].sel(B=B, theta=theta).values * 1e3 # Convert to GHz
amps = xarr['amplitudes'].sel(B=B, theta=theta).values

# Define the frequency axis for plotting
freq_min, freq_max = -3, 3
n_points = 8000
freq_axis = np.linspace(freq_min, freq_max, n_points)
gamma = 0.01  # MHz

spectrum = get_concoluted_spectrum(freq, amps, freq_axis, gamma)

# Plot
fig = create_ZF_base_figure()
fig.add_trace(go.Scatter(x=freq_axis, y=spectrum, mode='lines', line=dict(color='black', width=2)))
add_doubleheaded_arrow(fig, freq[1], freq[0], 0.45, r'$\Large A_\text{iso}$', label_offset=-12, font_size=20)
add_doubleheaded_arrow(fig, freq[2], freq[3], 0.45, r'$\Large A_\text{iso}$', label_offset=-12, font_size=20)

fig.show(renderer='browser')
# fig.write_image('../../Figures/Manuscript/TF_simulations/muSR_iso_ZFS.pdf')


### EPR zero field splitting


In [ ]:
xarr = xr.load_dataset('Data/TF_EPR_dipolar_cc.nc')
# print(ds)
theta = 0
B = 0

freq = xarr['frequencies'].sel(B=B, theta=theta).values * 1e3 # Convert to GHz
amps = xarr['amplitudes'].sel(B=B, theta=theta).values

# Define the frequency axis for plotting
freq_min, freq_max = -2, 2
n_points = 8000
freq_axis = np.linspace(freq_min, freq_max, n_points)
gamma = 0.01  # MHz

spectrum = get_concoluted_spectrum(freq, amps, freq_axis, gamma)

# Plot
fig = create_ZF_base_figure()
fig.add_trace(go.Scatter(x=freq_axis, y=spectrum, mode='lines', line=dict(color='black', width=2)))
add_doubleheaded_arrow(fig, freq[0], freq[2], 0.8, r'$\Large \frac{1}{2}D_\parallel$', label_offset=-2, font_size=20)

fig.show(renderer='browser')
# fig.write_image('../../Figures/Manuscript/TF_simulations/EPR_dipolar_ZFS.pdf')


# Avoided level-crossing
## Analyitical model

In [ ]:
def muon_polarization_time_integrated(magnetic_fields, theta, Pz_0, off_resonance_state_rate, A_iso, D_parallel):
    """
    Calculate the time-integrated muon polarization for a given magnetic field and angle theta.
    """
    gamma_muon = 135.5 # in MHz/T
    inv_muon_lifetime = 0.4551 # in MHz
    osc_relaxation = inv_muon_lifetime + off_resonance_state_rate # in MHz

    def calc_q(theta, D_parallel):
        return 0.75 * D_parallel * np.sin(theta) * np.cos(theta)

    def calc_nu_mu(A_iso, D_parallel, theta):
        return (A_iso + D_parallel/2 * (3 * np.cos(theta)**2 - 1)) / 2

    q = calc_q(theta, D_parallel)
    nu_mu = calc_nu_mu(A_iso, D_parallel, theta)
    nu_mu0 = gamma_muon * magnetic_fields

    nominator = 0.5 * q**2 * Pz_0
    denominators = (osc_relaxation/2/np.pi)**2 + q**2 + (nu_mu0 - nu_mu)**2

    return 1 - nominator/denominators

magnetic_fields = np.linspace(1.85, 1.95, 3000) # in Tesla
thetas = np.radians([1, 5, 20, 45, 70, 85, 89]) # given in degrees and converted to radians
Pz_0 = 1.0
off_resonance_state_rate = 0
A_iso = 514.8 # in MHz
D_parallel = 2 # in MHz

df = pd.DataFrame({'B / T': magnetic_fields})
for theta in thetas:
    df[f"θ = {np.degrees(theta):.0f}°"] = muon_polarization_time_integrated(magnetic_fields, theta, Pz_0, off_resonance_state_rate, A_iso, D_parallel)

thetas_powder = np.radians(np.linspace(0, 90, 500))
polarization_matrix = np.column_stack([
    muon_polarization_time_integrated(magnetic_fields, theta, Pz_0, off_resonance_state_rate, A_iso, D_parallel)
    for theta in thetas_powder
])

df_powder = pd.DataFrame(polarization_matrix, columns=[f"θ_{i}" for i in range(len(thetas_powder))])
df_powder.insert(0, "B", magnetic_fields)

# weighted mean across thetas
weights = np.sin(thetas_powder)
polarizations_weighted = np.average(polarization_matrix, axis=1, weights=weights)

df["Powder"] = polarizations_weighted

# Color handling + plotting
# TODO: decide on best color scheme for different angles
pastel = pc.qualitative.Set2
colors = pastel[:len(thetas)] + ["black"]  # auto-adjust to number of angles + powder avg

fig = px.line(df, x="B / T", y=df.columns[1:], color_discrete_sequence=colors)
fig.update_layout(height=400, width=600,
                  xaxis_title='B / T',
                  xaxis_range=[1.875, 1.935],
                  # yaxis_title=r"$\Large \left < P_z \right >$",  # _z is a bit ugly like this
                  yaxis_title='<i>P<sub>z</sub></i>',
                  legend=dict(y=0.015, yanchor="bottom", title=''),
                  )
fig.show()
# fig.write_image('../../Figures/Manuscript/ALC/ALC_analytical.pdf')

## Numerical model using Spidyan

In [ ]:
num_data_angles = loadmat('../../MatLab/ALCvsMQ/Data/num_ALC_simulation_thetas.mat')
num_data_powder = loadmat('../../MatLab/ALCvsMQ/Data/num_ALC_simulation_powder.mat')
thetas = [1, 5, 20, 45, 70, 85, 89]  # thetas of the single orientation spectra

spectra = num_data_angles['spectra'][:, 0]  # shape (n_B, n_thetas) (0 is for O = Ix)

col_names = [f"θ = {theta}°" for theta in thetas]
df = pd.DataFrame(spectra.transpose(), columns=col_names)
magnetic_fields = num_data_angles['magnetic_fields'].flatten()  # shape (n_B,)

# powder average
# df['Powder'] = num_data_powder['powder_spectrum']

pastel = pc.qualitative.Set2
colors = pastel[:len(thetas)] + ["black"]  # auto-adjust to number of angles + powder avg

fig = go.Figure()

# Use normal graph_objects as the dashed line is not added in the foreground for some stupid reason
for col, color in zip(df.columns, colors):
    fig.add_trace(go.Scatter(
        x=magnetic_fields,
        y=df[col],
        mode="lines",
        line=dict(color=color),
        name=col
    ))
fig.add_trace(go.Scatter(x=num_data_powder['magnetic_fields'].flatten(),
                         y=num_data_powder['powder_spectrum'].flatten(),
                         mode='lines',
                         line=dict(color='red', width=2, dash='dash'),
                         name='Powder'))

fig.update_layout(height=400, width=600,
                  xaxis_title='B / T',
                  xaxis_range=[1.865, 1.925],
                  # yaxis_title=r"$\Large \left < P_z \right >$",  # _z is a bit ugly like this
                  yaxis_title='<i>P<sub>z</sub></i>',
                  legend=dict(y=0.015, yanchor="bottom", title=''),
                  )
fig.show()
# fig.write_image('../../Figures/Manuscript/ALC/ALC_numerical.pdf')

#### Comparison of numerical and analytical powder average

In [ ]:
x_num = num_data_powder['magnetic_fields'].flatten()
y_num = num_data_powder['powder_spectrum'].flatten()

magnetic_fields = x_num
Pz_0 = 1.0
off_resonance_state_rate = 0
A_iso = 514.8 # in MHz
D_parallel = 2 # in MHz

thetas_powder = np.radians(np.linspace(0, 90, 500))
polarization_matrix = np.column_stack([
    muon_polarization_time_integrated(magnetic_fields, theta, Pz_0, off_resonance_state_rate, A_iso, D_parallel)
    for theta in thetas_powder
])

df_powder = pd.DataFrame(polarization_matrix, columns=[f"θ_{i}" for i in range(len(thetas_powder))])
df_powder.insert(0, "B", magnetic_fields)

# weighted mean across thetas
weights = np.sin(thetas_powder)
y_ana = np.average(polarization_matrix, axis=1, weights=weights)

x_ana = magnetic_fields

shift_peak = x_ana[y_ana.argmin()] - x_num[y_num.argmin()]
print(f'Shift of peaks: {shift_peak}')

x_num_shifted = x_num + shift_peak

fig = go.Figure()
# Analytical
fig.add_trace(go.Scatter(x=x_ana, y=y_ana,
                         mode='lines',
                         line=dict(color='black', width=2),
                         name='Analytical'))
# Numerical
fig.add_trace(go.Scatter(x=x_num_shifted, y=y_num,
                         mode='lines',
                         line=dict(color='red', width=2, dash='dash'),
                         name='Numerical'))
fig.update_layout(height=400, width=600,
                  xaxis_title='B / T',
                  xaxis_range=[1.87, 1.93],
                  yaxis_title='<i>P<sub>z</sub></i>',
                  yaxis=dict(
                      tick0=0.7,
                      dtick=0.1,
                      range=[0.68, 1],
                  ),
                  legend=dict(y=0.015, yanchor="bottom", title=''),
                  )
# fig.write_image('../../Figures/Manuscript/ALC/ALC_analytical_vs_numerical_powder_average.pdf')
fig.show()

Shift between numerical and analytical model for each theta

In [ ]:
df = pd.read_csv('../../MatLab/ALCvsMQ/ana_vs_num_peak_positions.csv')
print(np.mean(df['peak_pos_diff'])*1000)  # in mT

fig = go.Figure()
fig.add_trace(go.Scatter(x=df['theta'], y=df['peak_pos_diff']*1000,  # convert to mT
                         mode='lines',
                         line=dict(color='black', width=2),))
fig.update_layout(height=400, width=600,
                  xaxis_title='θ / °',
                  yaxis_title='ΔB / mT',
                  # xaxis_range=[1.87, 1.93],
                  )
# fig.write_image('../../Figures/Manuscript/ALC/ALC_analytical_vs_numerical_peak_position_difference.pdf')
fig.show()

### Time traces for numerical model

In [ ]:
# awg.s_rate = 20, theta = 45 deg, A_iso = 514.8 MHz, D_parallel = 2 MHz, T1 = T2 = 2.2 us
# On resonance case (at B = 1.8922 T)
df_on = loadmat('../../MatLab/ALCvsMQ/Data/ALC_signal_time_evolution_on_resonance.mat')
print(df_on)
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_on['time_ds'].flatten()/1000,  # convert to microseconds))
                         y=np.real(df_on['trace'].flatten()),
                         mode='lines',
                         line=dict(color='black', width=2),
                         ))
fig.update_layout(height=400, width=600,
                  xaxis_range=[0, 12],
                  yaxis_title='<i>P<sub>z</sub></i>',
                  xaxis_title='Time / μs',
                  margin=dict(b=80),
                  xaxis=dict(
                      title_standoff=8,
                  ),
                  yaxis=dict(
                      tick0=0.2,
                      dtick=0.2,
                  )
                 )
# fig.write_image('../../Figures/Manuscript/ALC/numerical_ALC_time_trace_on_resonance.pdf')
fig.show()

# Off resonance case (at B = 1.82 T)
df_off = loadmat('../../MatLab/ALCvsMQ/Data/ALC_signal_time_evolution_off_resonance.mat')

fig = go.Figure()
fig.add_trace(go.Scatter(x=df_off['time_ds'].flatten()/1000,  # convert to microseconds
                         y=np.real(df_off['trace'].flatten()),
                         mode='lines',
                         line=dict(color='black', width=2),
                         ))
fig.update_layout(height=400, width=600,
                  xaxis_range=[0, 12],
                  xaxis_title='Time / μs',
                  margin=dict(b=80),
                  xaxis=dict(
                      title_standoff=8,
                  ),
                  yaxis_title='<i>P<sub>z</sub></i>',
                  yaxis=dict(
                      tick0=0.994,
                      dtick=0.002,
                  )
                 )
# fig.write_image('../../Figures/Manuscript/ALC/numerical_ALC_time_trace_off_resonance.pdf')
fig.show()

## Repolarization
Plot of muon repolarization using simple analytical model

In [ ]:
def muon_repolarization(magnetic_fields, A_iso, gamma_electron=-28024.95, gamma_muon=135.5):
    B = A_iso / (gamma_electron + gamma_muon)
    temps = [(magnetic_field / B)**2 for magnetic_field in magnetic_fields]
    return [(1+2*temp)/(2*(1+temp)) for temp in temps]

magnetic_fields = np.linspace(0, 1, 2000) # in Tesla
A_iso = 514.8 # in MHz

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=np.log10(magnetic_fields),
    # x=magnetic_fields,
    y=muon_repolarization(magnetic_fields, A_iso),
    mode='lines',
    line=dict(color='black', width=2),
))
fig.update_layout(
    height=400, width=600,
    xaxis_title='log<sub>10</sub>(B / T)',
    xaxis=dict(
        title_standoff=8,
    ),
    yaxis_title='<i>P<sub>z</sub></i>',
    margin=dict(l=90, r=20, t=20, b=85),

)
fig.show()
# fig.write_image('../../Figures/Manuscript/ALC/muon_repolarization_analytical.pdf')

# Pake Pattern for powder sample


# Energy level splitting as a function of B

In [ ]:
df = pd.read_csv('Data/energy_levels_A_iso_1GHz.csv', index_col=0)

magnetic_fields = df.index.values
fig = go.Figure()
for col in df.columns:
    fig.add_trace(go.Scatter(x=magnetic_fields, y=df[col],
                             mode='lines',
                             line=dict(width=2),
                             showlegend=False,
                             ))
fig.update_layout(height=400, width=600,
                  xaxis_title='B',
                  yaxis_title='Energy'
                 )

# Add state labels
texts = ['|3⟩', '|4⟩', '|2⟩', '|1⟩']
x_position = df.index[380]  # in T
y_at_x = df.loc[x_position].values
print(y_at_x)
y_positions = [-1.68115714-0.2, -1.07806905+0.25,  1.18115714-0.2,  1.57806905+0.2]
colors = px.colors.qualitative.G10[:4]
for y, label, color in zip(y_positions, texts, colors):
    fig.add_annotation(
        x=x_position, y=y,
        text=label,
        showarrow=False,
        font=dict(size=20, color=color),
    )

fig.update_layout(
    xaxis=dict(
        showticklabels=False,   # remove x ticks
        showgrid=False,
        ticks='',               # remove tick marks
        title_standoff=8,        # decrease distance between x label and axis
        minor=dict(ticks='', showgrid=False),  # remove minor ticks and minor grid
    ),
    yaxis=dict(
        showticklabels=False,   # remove x ticks
        showgrid=False,
        ticks='',               # remove tick marks
        title_standoff=8,        # decrease distance between x label and axis
        minor=dict(ticks='', showgrid=False),  # remove minor ticks and minor grid
    ),
)

fig.show()